# Combustion Model Summary for Common Textile Fibers

| **Fiber**           | **Chemical Nature**         | **Repeat Unit Formula** | **Molar Mass (g/mol)** | **Notable Combustion Products**       |
|---------------------|-----------------------------|--------------------------|------------------------|----------------------------------------|
| **Cotton**          | Natural cellulose           | C₆H₁₀O₅                 | 162                    | CO₂, H₂O                               |
| **Wool**            | Natural protein (keratin)   | C₂H₅NO + trace S         | ~75                   | CO₂, H₂O, NOₓ, SO₂                      |
| **Silk**            | Natural protein (fibroin)   | C₁₅H₂₀N₅O₆              | 365                    | CO₂, H₂O, NOₓ                           |
| **Viscose**         | Regenerated cellulose       | C₆H₁₀O₅                 | 162                    | CO₂, H₂O                               |
| **Polyester (PET)** | Synthetic polymer           | C₁₀H₈O₄                 | 192                    | CO₂, H₂O, CO, soot                      |
| **Polyamide/Nylon** | Synthetic polymer           | C₁₂H₂₂N₂O₂              | 226                    | CO₂, H₂O, NOₓ                           |
| **Acrylic (PAN)**   | Synthetic polymer           | C₃H₃N                   | 53                     | CO₂, H₂O, HCN                           |

> Notes:
> - All combustion reactions assume **complete oxidation**.
> - **Nitrogen-containing** fibers release NOₓ (e.g. wool, silk, nylon).
> - **Sulfur-containing** fibers (e.g. wool) produce SO₂.
> - **Acrylic** releases **hydrogen cyanide (HCN)** upon burning.


In [2]:

import bw2io as bi
import bw2calc as bc
import bw2data as bd
PROJECT_NAME = "Textile_SY16"
TAR_PATH     = '/etc/data/ecoinvent-3.10-cutoff-bw25.tar.gz'

# If the project already exists, just switch to it,
# otherwise restore from archive and then switch.
if PROJECT_NAME in bd.projects:
    bd.projects.set_current(PROJECT_NAME)
else:
    bi.restore_project_directory(
        fp=TAR_PATH,
        project_name=PROJECT_NAME,
    )
    bd.projects.set_current(PROJECT_NAME)


Restoring project backup archive - this could take a few minutes...
Restored project: Textile_SY16
02:31:29+0000 [info     ] Applying automatic update: 4.0 database search directories FTS5
02:31:29+0000 [info     ] Reindexing database ecoinvent-3.10-biosphere
02:31:29+0000 [info     ] Reindexing database ecoinvent-3.10-cutoff


In [3]:
from dataclasses import dataclass
from typing import Dict
import pprint
import bw2data as bd

@dataclass
class Fiber:
    name: str
    formula: Dict[str, int]  # Elemental composition per repeat unit
    molar_mass: float        # g/mol
    byproducts: Dict[str, int]  # Special combustion byproducts like HCN, SO2, NOx

# Define fibers
fibers = [
    Fiber("Cotton", {"C":6, "H":10, "O":5}, 162, {}),
    Fiber("Wool", {"C":2, "H":5, "O":1, "N":1, "S":0.05}, 75, {"SO2": 0.05, "NOx": 1}),
    Fiber("Silk", {"C":15, "H":20, "O":6, "N":5}, 365, {"NOx": 5}),
    Fiber("Viscose", {"C":6, "H":10, "O":5}, 162, {}),
    Fiber("Polyester", {"C":10, "H":8, "O":4}, 192, {}),
    Fiber("Polyamide/Nylon", {"C":12, "H":22, "O":2, "N":2}, 226, {"NOx": 2}),
    Fiber("Acrylic", {"C":3, "H":3, "N":1}, 53, {"HCN": 1}),
]

# Molar masses
atomic_weights = {
    "C": 12.01,
    "H": 1.008,
    "O": 16.00,
    "N": 14.01,
    "S": 32.06,
    "CO2": 44.01,
    "H2O": 18.02,
    "SO2": 64.06,
    "NOx": 46.00,  # Approx for NO2
    "HCN": 27.03,
}

def combustion_products(fiber: Fiber):
    moles = 1000 / fiber.molar_mass  # 1 kg of fiber
    products = {}

    # CO2: 1 per C atom
    if "C" in fiber.formula:
        products["CO2"] = moles * fiber.formula["C"]

    # H2O: 0.5 per H atom
    if "H" in fiber.formula:
        products["H2O"] = moles * fiber.formula["H"] / 2

    # NOx, SO2, HCN etc.
    for gas, count in fiber.byproducts.items():
        products[gas] = moles * count

    # Convert to mass (g)
    products_mass = {gas: round(mol * atomic_weights[gas] / 1000, 3) for gas, mol in products.items()}  # in kg

    return products_mass

# Display results
for fiber in fibers:
    mass_output = combustion_products(fiber)
    print(f"\n Combustion products from 1 kg of {fiber.name}:")
    pprint.pprint(mass_output)



 Combustion products from 1 kg of Cotton:
{'CO2': 1.63, 'H2O': 0.556}

 Combustion products from 1 kg of Wool:
{'CO2': 1.174, 'H2O': 0.601, 'NOx': 0.613, 'SO2': 0.043}

 Combustion products from 1 kg of Silk:
{'CO2': 1.809, 'H2O': 0.494, 'NOx': 0.63}

 Combustion products from 1 kg of Viscose:
{'CO2': 1.63, 'H2O': 0.556}

 Combustion products from 1 kg of Polyester:
{'CO2': 2.292, 'H2O': 0.375}

 Combustion products from 1 kg of Polyamide/Nylon:
{'CO2': 2.337, 'H2O': 0.877, 'NOx': 0.407}

 Combustion products from 1 kg of Acrylic:
{'CO2': 2.491, 'H2O': 0.51, 'HCN': 0.51}


In [4]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

In [5]:
biosphere = bd.Database("ecoinvent-3.10-biosphere")

In [7]:
# Example: search for common combustion byproducts
search_terms = ["carbon dioxide", "water", "sulfur dioxide", "nitrogen oxides", "hydrogen cyanide", "carbon monoxide", "soot"]

for term in search_terms:
    print(f"\n--- Results for: {term.upper()} ---")
    matches = [flow for flow in biosphere if term.lower() in flow['name'].lower()]
    
    if not matches:
        print("No matches found.")
        continue

    # for i, flow in enumerate(matches[:3]):  # Limit output to top 3
    #     print(f"\nMatch {i+1}:")
    #     print(f"  Name       : {flow['name']}")
    #     print(f"  Categories : {flow.get('categories', 'N/A')}")
    #     print(f"  Unit       : {flow.get('unit', 'N/A')}")
    #     print(f"  Type       : {flow.get('type', 'N/A')}")
    #     print(f"  Comment    : {flow.get('comment', 'No comment')[:300]}")


--- Results for: CARBON DIOXIDE ---

--- Results for: WATER ---

--- Results for: SULFUR DIOXIDE ---

--- Results for: NITROGEN OXIDES ---

--- Results for: HYDROGEN CYANIDE ---

--- Results for: CARBON MONOXIDE ---

--- Results for: SOOT ---
No matches found.


In [8]:
biosphere.search("water")

['Water' (cubic meter, None, ('water', 'surface water')),
 'Water' (cubic meter, None, ('water',)),
 'Water' (cubic meter, None, ('water', 'ocean')),
 'Water' (cubic meter, None, ('water', 'ground-')),
 'Water' (cubic meter, None, ('water', 'fossil well')),
 'Water' (cubic meter, None, ('water', 'ground-, long-term')),
 'Water, river' (cubic meter, None, ('natural resource', 'in water')),
 'Water, lake' (cubic meter, None, ('natural resource', 'in water')),
 'Water, salt, ocean' (cubic meter, None, ('natural resource', 'in water')),
 'Water, salt, sole' (cubic meter, None, ('natural resource', 'in water')),
 'Water, well, in ground' (cubic meter, None, ('natural resource', 'in water')),
 'Water, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water')),
 'Water, cooling, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water')),
 'Water' (cubic meter, None, ('air',)),
 'Water, turbine use, unspecified natural origin' (cubic meter, None, (

In [9]:
biosphere.search("carbon dioxide")

['Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')),
 'Carbon dioxide, from soil or biomass stock' (kilogram, None, ('air',)),
 'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil',)),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil', 'industrial')),
 'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil', 'forestry')),
 'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil', 'agricultural')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',

In [10]:
biosphere.search("sulfur dioxide")

['Sulfur dioxide' (kilogram, None, ('air',)),
 'Sulfur dioxide' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Sulfur dioxide' (kilogram, None, ('air', 'urban air close to ground')),
 'Sulfur dioxide' (kilogram, None, ('air', 'low population density, long-term')),
 'Sulfur dioxide' (kilogram, None, ('air', 'non-urban air or from high stacks'))]

In [11]:
biosphere.search("nitrogen oxides")

['Nitrogen oxides' (kilogram, None, ('air',)),
 'Nitrogen oxides' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Nitrogen oxides' (kilogram, None, ('air', 'urban air close to ground')),
 'Nitrogen oxides' (kilogram, None, ('air', 'low population density, long-term')),
 'Nitrogen oxides' (kilogram, None, ('air', 'non-urban air or from high stacks'))]

In [12]:
biosphere.search("hydrogen cyanide")

['Hydrogen cyanide' (kilogram, None, ('air',))]

In [13]:
biosphere.search("carbon monoxide")

['Carbon monoxide, fossil' (kilogram, None, ('air',)),
 'Carbon monoxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon monoxide, from soil or biomass stock' (kilogram, None, ('air',)),
 'Carbon monoxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon monoxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon monoxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon monoxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon monoxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon monoxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon monoxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon monoxide, from soil or biomass stock' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon monoxide, from soil or biomass stock' (kilogram, None

In [25]:
def get_biosphere_flow(name, unit=None, categories=None, database="ecoinvent-3.10-biosphere"):
    matches = [
        flow for flow in bd.Database(database)
        if flow.get('name') == name
        and (unit is None or flow.get('unit') == unit)
        and (categories is None or flow.get('categories') == categories)
    ]

    if len(matches) == 0:
        raise ValueError(f" No biosphere flow found for: {name} - {unit} - {categories}")
    elif len(matches) == 1:
        return matches[0]
    else:
        print(f"⚠️ Multiple matches found for: {name}")
        for i, match in enumerate(matches):
            print(f"  Match {i+1}: {match['name']} | {match.get('unit')} | {match.get('categories')}")
        raise ValueError(" Too many matches. Refine your query.")

In [35]:
element_flows = {}
# element_flows = {
#     "H2O": get_biosphere_flow("Water, in air", "cubic meter", ("natural resource", "in air")),
#     "CO2": get_biosphere_flow("Carbon dioxide, non-fossil", "kilogram", ("air",)),
#     "NOx": get_biosphere_flow("Nitrogen oxides", "kilogram", ("air",)),
#     "SO2": get_biosphere_flow("Sulfur dioxide", "kilogram", ("air",)),
#     "HCN": get_biosphere_flow("Hydrogen cyanide", "kilogram", ("air",)),
#     "CO": get_biosphere_flow("Carbon monoxide, fossil", "kilogram", ("air",)),
# }
element_flows = {
    "H2O": get_biosphere_flow("Water, in air", "cubic meter", ("natural resource", "in air")),
    "CO2": get_biosphere_flow("Carbon dioxide, fossil", "kilogram", ("air",)),
    "NOx": get_biosphere_flow("Nitrogen oxides", "kilogram", ("air",)),
    "SO2": get_biosphere_flow("Sulfur dioxide", "kilogram", ("air",)),
    "CO": get_biosphere_flow("Carbon monoxide, fossil", "kilogram", ("air",)),
    # "HCN": optional — most LCIA methods don’t support this
}

In [36]:
fiber_db = bd.Database("fiber_combustion")
activity_data = {}

for fiber in fibers:
    outputs = combustion_products(fiber)

    exchanges = []
    for elem, amount in outputs.items():
        flow = element_flows.get(elem)
        if not flow:
            print(f"⚠️ {elem} not found — skipping in {fiber.name}")
            continue
        exchanges.append({
            'input': flow.key,
            'amount': amount,
            'type': 'biosphere',
            'unit': flow['unit'],
            'name': flow['name'],
        })

    exchanges.append({
        'input': ("fiber_combustion", f"{fiber.name.lower()}_combustion"),
        'amount': 1.0,
        'type': 'production',
        'unit': "kilogram",
        'name': f"Combustion of {fiber.name}",
    })

    activity_data[("fiber_combustion", f"{fiber.name.lower()}_combustion")] = {
        'name': f"Combustion of {fiber.name}",
        'unit': "kilogram",
        'exchanges': exchanges,
    }

fiber_db.write(activity_data)
print("✅ Regenerated all activities with updated flows.")

⚠️ HCN not found — skipping in Acrylic
03:02:12+0000 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 7/7 [00:00<00:00, 43755.78it/s]

03:02:12+0000 [info     ] Vacuuming database            


✅ Regenerated all activities with updated flows.


In [48]:
fiber_activities = {
    fiber.name: bd.get_activity(("fiber_combustion", f"{fiber.name.lower()}_combustion"))
    for fiber in fibers
}

methods = [
    ('IPCC 2013', 'climate change', 'global warming potential (GWP100)'),  # ✅ This works on your system
    ('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP100)'),  # ✅ This too
]

results = np.zeros((len(fiber_activities), len(methods)))
products = list(fiber_activities.values())

for i, product in enumerate(products):
    for j, method in enumerate(methods):
        try:
            lca = bc.LCA({product: 1}, method=method)
            lca.lci()
            lca.lcia()
            results[i, j] = lca.score
        except Exception as e:
            print(f"⚠️ Error in {product['name']} with {method[1]}: {e}")
            results[i, j] = np.nan

df_results = pd.DataFrame(
    results,
    index=[p['name'] for p in products],
    columns=[f"{m[0]} - {m[2]}" for m in methods]
)

print("\n📊 LCIA Results:")
print(df_results)



📊 LCIA Results:
                               IPCC 2013 - global warming potential (GWP100)  \
Combustion of Cotton                                                   1.630   
Combustion of Wool                                                     1.174   
Combustion of Silk                                                     1.809   
Combustion of Viscose                                                  1.630   
Combustion of Polyester                                                2.292   
Combustion of Polyamide/Nylon                                          2.337   
Combustion of Acrylic                                                  2.491   

                               ReCiPe 2016 v1.03, midpoint (H) - global warming potential (GWP100)  
Combustion of Cotton                                                       1.630                    
Combustion of Wool                                                         1.174                    
Combustion of Silk                     

In [49]:
# for m in bd.methods:
#     print(m)


In [50]:
# for m in bd.methods:
#     if "climate" in m[1].lower():
#         print(m)